In [ ]:
import argparse
import copy
import logging
import os
import sys
import traceback
from datetime import datetime
from flask import Flask, json, request, Response
from flask_restful import Resource, Api
from indigo import Indigo
from typing import List
import pandas as pd

sys.argv=['']

base_response = {
    "status": "FAIL",
    "error": "",
    "results": []
}


def parse_args():
    parser = argparse.ArgumentParser("atom_map_server")
    parser.add_argument("--server_ip", help="Server IP to use", type=str, default="0.0.0.0")
    parser.add_argument("--server_port", help="Server port to use", type=int, default=9919)
    parser.add_argument("--log_file", help="Log file", type=str, default="indigo_server")

    return parser.parse_args()


class IndigoAPI:
    def __init__(self):
        self.indigo = Indigo()

    def atom_map(self, smis: List[str], mode: str = "discard") -> List[str]:
        mapped_rxns = []
        for smi in smis:
            rxn = self.indigo.loadReaction(smi)
            rxn.automap(mode=mode)
            mapped_rxn = rxn.smiles()
            mapped_rxns.append(mapped_rxn)

        return mapped_rxns


class RxnMapperService(Resource):
    def post(self):
        response = copy.deepcopy(base_response)

        # request parsing
        try:
            request_json = request.get_json()
            smis = request_json["smiles"]
            assert(isinstance(smis, list) and smis)

        except Exception as e:
            traceback.print_tb(e.__traceback__)
            response["error"] = "parse request error, make sure to provide a list of reaction SMILES for 'smiles' field"

            return Response(json.dumps(response), mimetype="application/json")

        # mapping
        try:
            results = indigo_api.atom_map(smis)
            response["results"] = results
            response["status"] = "SUCCESS"
            return Response(json.dumps(response), mimetype="application/json")

        except Exception as e:
            traceback.print_tb(e.__traceback__)
            response["error"] = f"error during encoding, traceback: {traceback.print_tb(e.__traceback__)}"

            return Response(json.dumps(response), mimetype="application/json")


def test_indigo_api(indigo_api):
    df=pd.read_csv('炸药合成最新版.csv',encoding="gbk")
    df['Rxn']=df['Reactant']+'>>'+df['Product']
    results = indigo_api.atom_map(df['Rxn'])
    logging.info(results)
    dataframe = pd.DataFrame({'Reactant':df['Reactant'],'Product':df['Product'],'Rxn':df['Rxn'],'Rxnmapped':results})
    dataframe.to_csv('炸药合成.csv',index=False)


def start_server():
    print("Server starting..")

    app = Flask(__name__)
    api = Api(app)

    api.add_resource(RxnMapperService, "/indigo_mapper")

    app.run(host=args.server_ip, port=args.server_port, threaded=True)


if __name__ == "__main__":
    args = parse_args()

    # logger setup
    os.makedirs(f"./logs", exist_ok=True)
    dt = datetime.strftime(datetime.now(), "%y%m%d-%H%Mh")

    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    fh = logging.FileHandler(f"./logs/{args.log_file}.{dt}.log")
    sh = logging.StreamHandler(sys.stdout)
    fh.setLevel(logging.INFO)
    sh.setLevel(logging.INFO)
    logger.addHandler(fh)
    logger.addHandler(sh)

    # set up apis
    indigo_api = IndigoAPI()
    test_indigo_api(indigo_api)

    # start running
    start_server()
